In [ ]:
import hmftpy as hmf
import numpy as np
import matplotlib.pyplot as plt
from hmftpy.plaquettes.square import plaq4
from hmftpy.plaquettes.triangular import plaq3, plaq12
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from tqdm import tqdm

# Dzyaloshinskii–Moriya interaction
Here, we consider the DM interaction
\begin{align*}
    H_{DM} =&\vec D \cdot \sum_{\langle i, j\rangle} \vec \sigma_i \times\vec \sigma_j\\
    =&     D_x\sum_{\langle i, j \rangle}\left(\sigma_i^y\sigma_j^z - \sigma_i^z \sigma_j^y\right)
    +     D_y\sum_{\langle i, j \rangle} \left(\sigma_i^z\sigma_j^x - \sigma_i^x \sigma_j^z\right)
    +     D_z\sum_{\langle i, j \rangle} \left(\sigma_i^x\sigma_j^y - \sigma_i^y \sigma_j^x\right).
\end{align*}
For now, let's choose $D_z = 1$, $D_x=D_y=0$

In [ ]:
basis = spin_basis_1d(3, pauli=0) # pauli=0 fixes the operators as spin-1/2, rather than pauli matrices
interactions = {'nearest': {'xy': 1, 'yx': -1}}
Hi = hmf.operators.inner_hamiltonian(plaq3, interactions, basis, every_other=True, checks=True)
e, v = Hi.eigsh(k=1, which='SA')
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq3, interactions, basis, every_other=True, checks=True)
e, v = Hp.eigsh(k=1, which='SA')
ep = e[0]
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq3, interactions, basis, every_other=True)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

I want to check my work. By setting `every_other=True`, I restrict the sums to
\begin{equation}
\sum_i \sum_{j\in \mathcal N(i)} \rightarrow \sum_i \sum_{j\in \mathcal N(i), j < i}
\end{equation}
where $\mathcal N(i)$ is the list of neighbors of $i$. Without this restriction, the DMI terms cancel due to antisymmetry of the cross product.

On just the three-site triangle with open BC, the interaction in the first case (with `every_other=False`) is
\begin{equation}
(\sigma_0^x\sigma_1^y - \sigma_0^y \sigma_1^x)
+ (\sigma_0^x\sigma_2^y - \sigma_0^y \sigma_2^x)
+ (\sigma_1^x\sigma_0^y - \sigma_1^y \sigma_0^x)
+ (\sigma_1^x\sigma_2^y - \sigma_1^y \sigma_2^x)
+ (\sigma_2^x\sigma_0^y - \sigma_2^y \sigma_0^x)
+ (\sigma_2^x\sigma_1^y - \sigma_2^y \sigma_1^x)=0.
\end{equation}
With the restricted sum, I get
\begin{equation}
(\sigma_1^x\sigma_0^y - \sigma_1^y \sigma_0^x)
+ (\sigma_2^x\sigma_0^y - \sigma_2^y \sigma_0^x)
+ (\sigma_2^x\sigma_1^y - \sigma_2^y \sigma_1^x).
\end{equation}
If I instead sum bonds with some ordering, say $\{0,1\}$, $\{1,2\}$, $\{2,0\}$, I get
\begin{equation}
(\sigma_0^x\sigma_1^y - \sigma_0^y \sigma_1^x)
+ (\sigma_1^x\sigma_2^y - \sigma_1^y \sigma_2^x)
+ (\sigma_2^x\sigma_0^y - \sigma_2^y \sigma_0^x),
\end{equation}
which is not the same thing, since the terms are the same but the signs are not. What results do we get from this case?

In [ ]:
dm_lst = [['xy', [[1, 0, 1], [1, 1, 2], [1, 2, 0]]],
          ['yx', [[-1, 0, 1], [-1, 1, 2], [-1, 2, 0]]]
         ]
H_dm = quantum_operator({'static': dm_lst}, basis=basis)
dm_lst = [['xy', [[1, 0, 1], [1, 1, 2], [1, 2, 0]]],
          ['yx', [[-1, 0, 1], [-1, 1, 2], [-1, 2, 0]]]
         ]
e, v = H_dm.eigh()
print(e[0])

To fix this inconsistency, I added an additional way of forming Hamiltonians. For some clusters, I have added lists of nearest neighbor bonds with some consistently-chosen direction (counter-clockwise in upright triangles in these cases). This seems to give equivalent results to the previous case (skipping every-other bond)

In [ ]:
basis = spin_basis_1d(3, pauli=0) # pauli=0 fixes the operators as spin-1/2, rather than pauli matrices
interactions = {'x_bonds': {'xy': 1, 'yx': -1}, 'y_bonds': {'xy': 1, 'yx': -1}, 'z_bonds': {'xy': 1, 'yx': -1}}
Hi = hmf.operators.inner_hamiltonian(plaq3, interactions, basis, checks=True)
e, v = Hi.eigsh(k=1, which='SA')
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq3, interactions, basis, checks=True)
e, v = Hp.eigsh(k=1, which='SA')
ep = e[0]
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq3, interactions, basis)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

In [ ]:
basis = spin_basis_1d(12, pauli=0) # pauli=0 fixes the operators as spin-1/2, rather than pauli matrices
interactions = {'nearest': {'xy': 1, 'yx': -1}}
Hi = hmf.operators.inner_hamiltonian(plaq12, interactions, basis, every_other=True, checks=True)
e, v = Hi.eigsh(k=1, which='SA')
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq12, interactions, basis, every_other=True, checks=True)
e, v = Hp.eigsh(k=1, which='SA')
ep = e[0]
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq12, interactions, basis, every_other=True)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

interactions = {'x_bonds': {'xy': 1, 'yx': -1}, 'y_bonds': {'xy': 1, 'yx': -1}, 'z_bonds': {'xy': 1, 'yx': -1}}
Hi = hmf.operators.inner_hamiltonian(plaq12, interactions, basis, checks=True)
e, v = Hi.eigsh(k=1, which='SA')
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq12, interactions, basis, checks=True)
e, v = Hp.eigsh(k=1, which='SA')
ep = e[0]
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq12, interactions, basis)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))